## Useful Links
- http://www.cpcb.gov.in/CAAQM/frmUserAvgReportCriteria.aspx
- http://www.cpcb.gov.in/CAAQM/mapPage/frmindiamap.aspx

### Make Use of Beautiful Soup to automate fetching data
- Historic Annual PM2.5, PM10 Plot
    - INDIA - National
    - Delhi NCR
    - Delhi Location wise
    - Gurgaon

###Mechanize Links:
   - http://www.search.sourceforge.net/mechanize/
   - Forms Example: http://www.search.sourceforge.net/mechanize/forms.html
   - BSoup Input Parsing: http://stackoverflow.com/questions/23001678/python-beautiful-soup-form-input-parsing

In [1]:
from bs4 import BeautifulSoup
import mechanize
import re

In [2]:
url = 'http://www.cpcb.gov.in/CAAQM/frmUserAvgReportCriteria.aspx'

In [3]:
br = mechanize.Browser()
br.set_handle_robots(False)
r1 = br.open(url)
# print r1.read()  # body
forms = mechanize.ParseResponse(r1, backwards_compat=False)
form = forms[0]
# print form

form["ddlState"] = ["1",]
r1 = br.open(form.click())
forms = mechanize.ParseResponse(r1, backwards_compat=False)
form = forms[0]

form["ddlCity"] = ["21",]
r1 = br.open(form.click())
forms = mechanize.ParseResponse(r1, backwards_compat=False)
form = forms[0]

form["ddlStation"] = ["96",]
r1 = br.open(form.click())
forms = mechanize.ParseResponse(r1, backwards_compat=False)
form = forms[0]

form["lstBoxChannelLeft"] = ["1715",]
r1 = br.open(form.click())
forms = mechanize.ParseResponse(r1, backwards_compat=False)
form = forms[0]

form["ddlCriteria"] = ["0",]
r1 = br.open(form.click())
forms = mechanize.ParseResponse(r1, backwards_compat=False)
form = forms[0]

form["txtYear"] = "2013"
form["txtYearTo"] = "2017"
r1 = br.open(form.click())
forms = mechanize.ParseResponse(r1, backwards_compat=False)
form = forms[0]

form["btnSubmit"] = "True"
r1 = br.open(form.click())
br.select_form(nr=0)
r1 = br.submit()

In [83]:
def getOptions(param, form):
    return re.findall("{}=\[(.+)\]".format(param), str(form))

#     Example Usage: 
#     soup = BeautifulSoup(html)
#     getMap(soup, "ddlState")
def getMap(soup, ty):
    select_node = soup.findAll('select', attrs={'name': ty})
    if not select_node:
        select_node = soup.findAll('select', attrs={'id': ty})
    print select_node
    option_map = {}
    if select_node:
        for option in select_node[0].findAll('option'):
            option_map[option['value']] = option.text
    return option_map

def getPM25Id(html):
    return re.findall("<option value=(.+)>PM2\.5(.+)</option>", str(html))
    
def getStations(ddlState, ddlCity):
    br = mechanize.Browser()
    br.set_handle_robots(False)
    res = br.open(url)
    forms = mechanize.ParseResponse(res, backwards_compat=False)
    form = forms[0]
    # State
    form["ddlState"] = ["{}".format(ddlState),]
    form, html = updateForm(br, form)
    # City
    form["ddlCity"] = ["{}".format(ddlCity),]
    form, html = updateForm(br, form)
    
    options = getOptions("ddlStation", form)
    options = options[0].split(',')[1:]
    options = map(lambda x: x.strip(), options)
    stations = options
    print "Stations for state = {} and city = {}:".format(ddlState, ddlCity), stations
    return stations

def getData():
    stations = getStations("6", "85")
    soups = []
    #
    for station in stations:
        br = mechanize.Browser()
        br.set_handle_robots(False)
        res = br.open(url)
        forms = mechanize.ParseResponse(res, backwards_compat=False)
        form = forms[0]
        # Delhi
        form["ddlState"] = ["6",]
        form, html = updateForm(br, form)
        # Delhi
        form["ddlCity"] = ["85",]
        form, html = updateForm(br, form)

        form["ddlStation"] = ["{}".format(station),]
        form, html = updateForm(br, form)
        
#         print html
        print getPM25Id(html)
        PM25Id = getPM25Id(html)[0][0][1:-1]
        print "PM2.5 Id =", PM25Id

        form["lstBoxChannelLeft"] = ["{}".format(PM25Id),]
        form, html = updateForm(br, form)

        form["ddlCriteria"] = ["0",]
        form, html = updateForm(br, form)

        form["txtYear"] = "2013"
        form["txtYearTo"] = "2017"
        form, html = updateForm(br, form)

        form["btnSubmit"] = "True"
        form, html = updateForm(br, form)

        br.select_form("form1")
        res = br.submit(name='btnSubmit')
        html = res.read()
        soup = BeautifulSoup(html, 'html.parser')
        soups.append(soup)
        if len(soups) >= 2:
            break
        
    return soups

def updateForm(br, form):
    res = br.open(form.click())
    html = res.get_data()
    forms = mechanize.ParseResponse(res, backwards_compat=False)
    form = forms[0]
    return form, html

def getValsHtml(table):
    data = []
    rows = table.find_all('tr')
    for row in rows:
        cols = row.find_all('td')
        cols = [ele.text.strip() for ele in cols]
        data.append([ele for ele in cols if ele]) # Get rid of empty values
    vals = re.findall("\d+\.\d+", data[0][0])[:-1]
    data = []
    for val in vals:
        yr = val[:4]
        data.append([yr, val[4:]])
    return data

In [84]:
getData()

Stations for state = 6 and city = 85: ['1', '2', '3', '4', '5', '6', '7', '45', '46', '47', '51', '52', '53', '54', '55', '58']
[('"506"', '(Dust (PM2.5))')]
PM2.5 Id = 506
[('"507"', '(PM2.5)')]
PM2.5 Id = 507


[\n<!DOCTYPE html PUBLIC "-//W3C//DTD XHTML 1.0 Transitional//EN" "http://www.w3.org/TR/xhtml1/DTD/xhtml1-transitional.dtd">\n\n<html xmlns="http://www.w3.org/1999/xhtml">\n<head><title>\r\n\t::CPCB Air Quality Report::\r\n</title><link href="style.css" rel="stylesheet" type="text/css"/></head>\n<body bottommargin="0" leftmargin="0" rightmargin="0" topmargin="0">\n<form action="frmReportdisplay.aspx" id="form1" method="post" name="form1">\n<input id="__VIEWSTATE" name="__VIEWSTATE" type="hidden" value="/wEPDwULLTE3MzI2Njk3ODcPFgQeClN0YXJ0SW5kZXgFATAeCXRhYmxlZGF0YTKxCAABAAAA/////wEAAAAAAAAADAIAAABOU3lzdGVtLkRhdGEsIFZlcnNpb249Mi4wLjAuMCwgQ3VsdHVyZT1uZXV0cmFsLCBQdWJsaWNLZXlUb2tlbj1iNzdhNWM1NjE5MzRlMDg5BQEAAAAVU3lzdGVtLkRhdGEuRGF0YVRhYmxlAwAAABlEYXRhVGFibGUuUmVtb3RpbmdWZXJzaW9uCVhtbFNjaGVtYQtYbWxEaWZmR3JhbQMBAQ5TeXN0ZW0uVmVyc2lvbgIAAAAJAwAAAAYEAAAAhAU8P3htbCB2ZXJzaW9uPSIxLjAiIGVuY29kaW5nPSJ1dGYtMTYiPz4NCjx4czpzY2hlbWEgeG1sbnM9IiIgeG1sbnM6eHM9Imh0dHA6Ly93d3cudzMub3JnLzIwMDEvWE1MU2NoZW1hIiB4

In [85]:
soups = getData()
for soup in soups:
    table = soup.find(id="gvReportStation")
    if table:
        data = getValsHtml(table)
        print data
    else:
        print "No Data available for this station"

Stations for state = 6 and city = 85: ['1', '2', '3', '4', '5', '6', '7', '45', '46', '47', '51', '52', '53', '54', '55', '58']
[('"506"', '(Dust (PM2.5))')]
PM2.5 Id = 506
[('"507"', '(PM2.5)')]
PM2.5 Id = 507
No Data available for this station
[[u'2.5', u''], [u'2.5', u''], [u'2016', u'216.32'], [u'2017', u'170.78'], [u'2.5', u''], [u'2.5', u''], [u'193.', u'55'], [u'2.5', u'']]


In [4]:
br.select_form("form1")
r1 = br.submit(name='btnSubmit')
html = r1.read()
soup = BeautifulSoup(html, 'html.parser')

In [6]:
vals = re.findall("\d+\.\d+", data[0][0])[:-1]
for val in vals:
    yr = val[:4]
    print yr, val[4:]

2016 21.87
2017 23.62
